In [1]:
from resnet18 import ResNet, ResidualBlock, data_loader
import torch
import torch.nn as nn
import torch.backends
import torch.backends.mps
import torch.mps

In [2]:
train_loader, valid_loader = data_loader(data_dir='./data',
                                         batch_size=64)

test_loader = data_loader(data_dir='./data',
                              batch_size=64,
                              test=True)

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

device = torch.device(device)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
num_classes = 10
num_epochs = 20
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [2, 2, 2, 2], num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)  

# Train the model
total_step = len(train_loader)

import gc
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del images, labels, outputs
        torch.mps.empty_cache() if torch.backends.mps.is_available() else torch.cuda.empty_cache()
        gc.collect()

    print ('Epoch [{}/{}], Loss: {:.4f}' 
                    .format(epoch+1, num_epochs, loss.item()))
                
        # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs
        
        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total)) 
        
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))

Epoch [1/20], Loss: 1.5818
Accuracy of the network on the 5000 validation images: 62.58 %
Epoch [2/20], Loss: 0.2829
Accuracy of the network on the 5000 validation images: 72.58 %
Epoch [3/20], Loss: 1.7526
Accuracy of the network on the 5000 validation images: 77.2 %
Epoch [4/20], Loss: 0.4768
Accuracy of the network on the 5000 validation images: 80.36 %
Epoch [5/20], Loss: 0.6882
Accuracy of the network on the 5000 validation images: 81.86 %
Epoch [6/20], Loss: 0.7605
Accuracy of the network on the 5000 validation images: 80.14 %
Epoch [7/20], Loss: 0.8274
Accuracy of the network on the 5000 validation images: 81.34 %
Epoch [8/20], Loss: 0.0354
Accuracy of the network on the 5000 validation images: 82.4 %
Epoch [9/20], Loss: 0.0877
Accuracy of the network on the 5000 validation images: 83.1 %
Epoch [10/20], Loss: 0.5032
Accuracy of the network on the 5000 validation images: 82.4 %
Epoch [11/20], Loss: 0.2578
Accuracy of the network on the 5000 validation images: 82.3 %
Epoch [12/20]

In [4]:
torch.save(model.state_dict(), "model/DadyNgoy.pth")

In [5]:
torch.save(model.state_dict(), "model/DeLaPaz.pth")